In [2]:
import os
os.getcwd()
os.chdir('D:\\PycharmProjects\\DS_ML_Self\\FSDS_NOV_deepCNNClassifier')
os.getcwd()

'D:\\PycharmProjects\\DS_ML_Self\\FSDS_NOV_deepCNNClassifier'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class TrainingConfig:
    root_dir: Path 
    trained_model_path: Path
    updated_base_model_path: Path 
    trainig_data: Path 
    params_epochs: int 
    params_batch_size: int 
    params_is_augmentation: bool
    params_image_size: list 
    
@dataclass
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path 

In [4]:
from deepClassifier.constant import * 
from deepClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf
import time

In [5]:
class ConfigurationManger:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.confing = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.confing.artifacts_root])
        
    def get_prepare_callback_config(self) ->PrepareCallbacksConfig:
        config = self.confing.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])
        
        prepare_callback_config= PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath),
        )
        
        return prepare_callback_config
    
    def get_trainig_config(self) ->TrainingConfig:
        training = self.confing.training
        prepare_base_model = self.confing.prepare_base_model
        params = self.params
        training_data = os.path.join(self.confing.data_ingestion.unzip_dir, "PetImages")
        create_directories([
            Path(training.root_dir)
        ])
        
        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            trainig_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )
        return training_config

In [6]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config 
        
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir= os.path.join(
            self.config.tensorboard_root_log_dir,
            f'tb_logs_at_{timestamp}',
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
        
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks,
        ]

In [7]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config 
        
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        
    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1. / 255,
            validation_split = 0.20,
        )
        
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation='bilinear',
        )
        
        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.trainig_data,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs
        )
        
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator
            
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.trainig_data,
            subset='training',
            shuffle=True,
            **dataflow_kwargs
        )
        
    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model.save(path)
        
    def train(self, callback_list:list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks = callback_list
        )
        
        self.save_model(path=self.config.trained_model_path,
                        model=self.model)

In [8]:
# !pip install scipy

In [9]:
try:
    config = ConfigurationManger()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
    training_config = config.get_trainig_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
except Exception as e:
    raise e


Found 4998 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
  12/1250 [..............................] - ETA: 3:26:39 - loss: 15.3668 - accuracy: 0.5000